In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ตั้งค่า seed สำหรับสุ่มตัวเลข
np.random.seed(42)

student_data = {
    'รหัสนักศึกษา': [f'64{str(i).zfill(3)}' for i in range(1, 21)],
    'ชื่อ': ['สมชาย', 'สมหญิง', None, 'สมศักดิ์', 'สมใจ', 'กานต์', 'กิตติ',
             'จิรา', 'ชนาธิป', 'ดารา', 'ธีระ', 'นภา', 'บุญมี', 'ปรีชา',
             'พิมพ์', 'ภูมิ', 'มานี', 'ยุพา', 'รัตนา', 'ลัดดา'],
    'อายุ': [20, 21, None, 22, 19, 150, 21, 20, 23, 21,  # 150 = outlier
            None, 22, 20, 21, 22, 19, 20, 21, None, 22],
    'เพศ': ['ชาย', 'หญิง', 'Male', 'ช', 'ญ', 'ชาย', 'หญิง', 'F', 'M', 'หญิง',
            None, 'ชาย', 'หญิง', 'ชาย', 'Female', 'ช', 'ญ', 'หญิง', 'ชาย', None],
    'คณะ': ['วิทยาศาสตร์', 'วิทย์', 'Science', 'คณะวิทยาศาสตร์', 'วิศวะ',
            'Engineering', 'วิศวกรรม', 'ศิลปศาสตร์', 'Arts', 'ศิลป์',
            None, 'วิทย์', 'วิศวฯ', 'Liberal Arts', 'Sci',
            'วิทยาศาสตร์', 'วิศวกรรมศาสตร์', None, 'ศิลปศาสตร์', 'Science'],
    'GPA': [3.21, 3.45, 3.67, -1, 3.89, 3.12, 4.5, 2.98, 3.34, 3.56,
            3.78, None, 3.23, 3.45, 3.67, 3.89, 3.01, 3.23, None, 3.45],
    'ส่วนสูง_cm': [170, 165, 175, 168, 160, 172, 158, 180, 163, 167,
                  None, 171, 169, 164, 162, 173, 166, 161, 174, 168],
    'น้ำหนัก_kg': [65, 58, None, 70, 52, 75, 48, 82, 55, 60,
                  68, None, 72, 54, 56, 78, 62, 50, 76, 64],
    'ชั่วโมงเรียน_ต่อสัปดาห์': [15, 20, 25, 10, 30, 12, 35, 18, 22, 28,
                              None, 16, 24, 19, 26, 14, 21, 27, None, 23]
}

In [3]:
# สร้าง DataFrame หลัก
df_std = pd.DataFrame(student_data)

In [4]:
# เพิ่มวันที่ลงทะเบียน (สำหรับ feature engineering)
base_date = datetime(2024, 1, 15)
df_std['วันที่ลงทะเบียน'] = [base_date + timedelta(days=np.random.randint(-30, 30))
                                for _ in range(20)]

In [5]:
### **หมายเหตุ** ไม่สามารถอ่านไฟล์จาก csv ได้ ไม่แน่ใจด้วยเหตุใด ลองแก้ไขแล้วหลายครั้ง ไม่ได้ผล จึงทำการใช้ข้อมูลดิบเพื่อทำการบ้านต่อเลยครับ

In [6]:
#แสดงข้อมูล
print("="*100)
print("ข้อมูลหลัก (Student DataSet)")
print("="*100)
print(df_std)
print("\n ปัญหาที่พบในข้อมูล:")
print(f"- ค่าหายทั้งหมด: {df_std.isnull().sum().sum()} ค่า")
print(f"- อายุผิดปกติ: {df_std[df_std['อายุ'] > 100]['อายุ'].values if any(df_std['อายุ'] > 100) else 'ไม่มี'}")
print(f"- GPA ผิดปกติ: {df_std[(df_std['GPA'] < 0) | (df_std['GPA'] > 4)]['GPA'].values if any((df_std['GPA'] < 0) | (df_std['GPA'] > 4)) else 'ไม่มี'}")
print(f"- รูปแบบการเขียนคณะ: {df_std['คณะ'].nunique()} แบบ")
print(f"- รูปแบบการเขียนเพศ: {df_std['เพศ'].nunique()} แบบ")

ข้อมูลหลัก (Student DataSet)
   รหัสนักศึกษา      ชื่อ   อายุ     เพศ             คณะ   GPA  ส่วนสูง_cm  \
0         64001     สมชาย   20.0     ชาย     วิทยาศาสตร์  3.21       170.0   
1         64002    สมหญิง   21.0    หญิง           วิทย์  3.45       165.0   
2         64003      None    NaN    Male         Science  3.67       175.0   
3         64004  สมศักดิ์   22.0       ช  คณะวิทยาศาสตร์ -1.00       168.0   
4         64005      สมใจ   19.0       ญ           วิศวะ  3.89       160.0   
5         64006     กานต์  150.0     ชาย     Engineering  3.12       172.0   
6         64007     กิตติ   21.0    หญิง        วิศวกรรม  4.50       158.0   
7         64008      จิรา   20.0       F      ศิลปศาสตร์  2.98       180.0   
8         64009    ชนาธิป   23.0       M            Arts  3.34       163.0   
9         64010      ดารา   21.0    หญิง           ศิลป์  3.56       167.0   
10        64011      ธีระ    NaN    None            None  3.78         NaN   
11        64012       นภา   22.0   

In [7]:
#แสดงข้อมูลวันที่ที่เพิ่มมา
df_std.head(20)

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-01-23
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.45,165.0,58.0,20.0,2024-02-05
2,64003,None,NaN,Male,Science,3.67,175.0,NaN,25.0,2024-01-13
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2023-12-30
4,64005,สมใจ,19.0,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-01-27
5,64006,กานต์,150.0,ชาย,Engineering,3.12,172.0,75.0,12.0,2023-12-23
6,64007,กิตติ,21.0,หญิง,วิศวกรรม,4.50,158.0,48.0,35.0,2024-01-05
7,64008,จิรา,20.0,F,ศิลปศาสตร์,2.98,180.0,82.0,18.0,2024-01-23
8,64009,ชนาธิป,23.0,M,Arts,3.34,163.0,55.0,22.0,2024-02-11
9,64010,ดารา,21.0,หญิง,ศิลป์,3.56,167.0,60.0,28.0,2024-01-03


In [8]:
# # เก็บสำเนาข้อมูลต้นฉบับ

In [9]:
df_original = df_std.copy()  # สำคัญ! ใช้ .copy() เพื่อไม่ให้กระทบข้อมูลต้นฉบับ

# 2.3 วิเคราะห์ข้อมูลเบื้องต้น (EDA)

In [10]:
# วิเคราะห์ข้อมูล df_std ที่สร้างไว้
print("="*50)
print("📋 ข้อมูลทั่วไปของ Dataset")
print("="*50)
df_std.info()

print("\n" + "="*50)
print("📊 สถิติเชิงพรรณนา")
print("="*50)
print(df_std.describe())

# ตรวจสอบค่าหายในแต่ละคอลัมน์
missing_counts = df_std.isnull().sum()
print("\n❌ ค่าหายในแต่ละคอลัมน์:")
for col, count in missing_counts[missing_counts > 0].items():
    percentage = (count / len(df_std)) * 100
    print(f"   - {col}: {count} ค่า ({percentage:.1f}%)")

📋 ข้อมูลทั่วไปของ Dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   รหัสนักศึกษา             20 non-null     object        
 1   ชื่อ                     19 non-null     object        
 2   อายุ                     17 non-null     float64       
 3   เพศ                      18 non-null     object        
 4   คณะ                      18 non-null     object        
 5   GPA                      18 non-null     float64       
 6   ส่วนสูง_cm               19 non-null     float64       
 7   น้ำหนัก_kg               18 non-null     float64       
 8   ชั่วโมงเรียน_ต่อสัปดาห์  18 non-null     float64       
 9   วันที่ลงทะเบียน          20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 1.7+ KB

📊 สถิติเชิงพรรณนา
             อายุ        GPA  ส่วนสูง_cm  น้ำหนัก

#3 เทคนิคการจัดการข้อมูลพื้นฐาน
## 3.1 การจัดการข้อมูลหาย

### วิธีที่ 1 ลบแถวที่มีค่าหาย

In [11]:
# แสดงข้อมูล
# เหมาะเมื่อมีข้อมูลหายน้อย - ลบแถวที่มีค่าหาย
df_clean = df_std.dropna()
df_clean

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-01-23
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.45,165.0,58.0,20.0,2024-02-05
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2023-12-30
4,64005,สมใจ,19.0,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-01-27
5,64006,กานต์,150.0,ชาย,Engineering,3.12,172.0,75.0,12.0,2023-12-23
6,64007,กิตติ,21.0,หญิง,วิศวกรรม,4.50,158.0,48.0,35.0,2024-01-05
7,64008,จิรา,20.0,F,ศิลปศาสตร์,2.98,180.0,82.0,18.0,2024-01-23
8,64009,ชนาธิป,23.0,M,Arts,3.34,163.0,55.0,22.0,2024-02-11
9,64010,ดารา,21.0,หญิง,ศิลป์,3.56,167.0,60.0,28.0,2024-01-03
12,64013,บุญมี,20.0,หญิง,วิศวฯ,3.23,169.0,72.0,24.0,2023-12-26


## 2.เติมค่าด้วยค่าเฉลี่ย

In [12]:
# แสดงข้อมูลก่อนเติมค่า
print("ก่อนเติมค่า")
print(df_std)

# เหมาะกับข้อมูลตัวเลข - ใช้ค่าเฉลี่ย
mean_age = df_std['อายุ'].mean()
df_std['อายุ'].fillna(mean_age, inplace=True)

print(f"\nค่าเฉลี่ยอายุ - {mean_age:.1f} ปี")
print("หลังเติมค่า")
print(df_std)
df_std

ก่อนเติมค่า
   รหัสนักศึกษา      ชื่อ   อายุ     เพศ             คณะ   GPA  ส่วนสูง_cm  \
0         64001     สมชาย   20.0     ชาย     วิทยาศาสตร์  3.21       170.0   
1         64002    สมหญิง   21.0    หญิง           วิทย์  3.45       165.0   
2         64003      None    NaN    Male         Science  3.67       175.0   
3         64004  สมศักดิ์   22.0       ช  คณะวิทยาศาสตร์ -1.00       168.0   
4         64005      สมใจ   19.0       ญ           วิศวะ  3.89       160.0   
5         64006     กานต์  150.0     ชาย     Engineering  3.12       172.0   
6         64007     กิตติ   21.0    หญิง        วิศวกรรม  4.50       158.0   
7         64008      จิรา   20.0       F      ศิลปศาสตร์  2.98       180.0   
8         64009    ชนาธิป   23.0       M            Arts  3.34       163.0   
9         64010      ดารา   21.0    หญิง           ศิลป์  3.56       167.0   
10        64011      ธีระ    NaN    None            None  3.78         NaN   
11        64012       นภา   22.0     ชาย           ว

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน
0,64001,สมชาย,20.000000,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-01-23
1,64002,สมหญิง,21.000000,หญิง,วิทย์,3.45,165.0,58.0,20.0,2024-02-05
2,64003,None,28.470588,Male,Science,3.67,175.0,NaN,25.0,2024-01-13
3,64004,สมศักดิ์,22.000000,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2023-12-30
4,64005,สมใจ,19.000000,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-01-27
5,64006,กานต์,150.000000,ชาย,Engineering,3.12,172.0,75.0,12.0,2023-12-23
6,64007,กิตติ,21.000000,หญิง,วิศวกรรม,4.50,158.0,48.0,35.0,2024-01-05
7,64008,จิรา,20.000000,F,ศิลปศาสตร์,2.98,180.0,82.0,18.0,2024-01-23
8,64009,ชนาธิป,23.000000,M,Arts,3.34,163.0,55.0,22.0,2024-02-11
9,64010,ดารา,21.000000,หญิง,ศิลป์,3.56,167.0,60.0,28.0,2024-01-03


## วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด
### ตัวอย่างข้อมูลนักศึกษาที่มีคณะหาย

In [13]:
l_data = {
    'ชื่อ' : ['ธีระ', 'ยุพา', 'สมชาย' , 'มานี' , 'บุญมี', 'สมศักดิ์' , 'ภูมิ', 'ลัดดา'],
    'คณะ' : ['None', 'None', 'วิศวกรรมศาสตร์', 'ศิลปศาสตร์', 'ศิลปศาสตร์', 'วิทยาศาสตร์' , 'วิทยาศาสตร์' , 'วิทยาศาสตร์']
}
df_std = pd.DataFrame(l_data)

# แปลง 'None' ให้เป็น NaN ก่อน
df_std['คณะ'] = df_std['คณะ'].replace('None', np.nan)

#หาค่าที่พบบ่อยสุด (mode)
most_common_faculty = df_std['คณะ'].mode()[0]
df_std['คณะ'].fillna(most_common_faculty, inplace=True)
df_std

,ชื่อ,คณะ
0,ธีระ,วิทยาศาสตร์
1,ยุพา,วิทยาศาสตร์
2,สมชาย,วิศวกรรมศาสตร์
3,มานี,ศิลปศาสตร์
4,บุญมี,ศิลปศาสตร์
5,สมศักดิ์,วิทยาศาสตร์
6,ภูมิ,วิทยาศาสตร์
7,ลัดดา,วิทยาศาสตร์


## วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด
### ตัวอย่างข้อมูลนักศึกษาที่มีเพศหาย

In [14]:

l_data = {
    'ชื่อ' : ['ธีระ', 'ยุพา', 'สมชาย' , 'มานี' , 'บุญมี', 'สมศักดิ์' , 'ภูมิ', 'ลัดดา' , 'สมใจ'],
    'เพศ' : ['None', 'หญิง', 'ชาย', 'หญิง', 'หญิง', 'ชาย' , 'ชาย' , 'None' , 'หญิง']
}
df_std = pd.DataFrame(l_data)

# แปลง 'None' ให้เป็น NaN ก่อน
df_std['เพศ'] = df_std['เพศ'].replace('None', np.nan)

#หาค่าที่พบบ่อยสุด (mode)
most_common_faculty = df_std['เพศ'].mode()[0]
df_std['เพศ'].fillna(most_common_faculty, inplace=True)
df_std

,ชื่อ,เพศ
0,ธีระ,หญิง
1,ยุพา,หญิง
2,สมชาย,ชาย
3,มานี,หญิง
4,บุญมี,หญิง
5,สมศักดิ์,ชาย
6,ภูมิ,ชาย
7,ลัดดา,หญิง
8,สมใจ,หญิง


## วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด
### ตัวอย่างข้อมูลนักศึกษาที่มีเกรดที่หายไป

In [15]:
l_data = {
    'ชื่อ' : ['ธีระ', 'ยุพา', 'สมชาย' , 'มานี' , 'บุญมี', 'ภูมิ', 'ลัดดา' , 'สมใจ' , 'นภา'],
    'GPA' : ['3.78', '3.23', '3.21', '3.01', '3.23' , '3.89' , '3.45' , '3.89' , 'nan' ]
}
df_std = pd.DataFrame(l_data)

# แปลงค่าในคอลัมน์ GPA ให้เป็นตัวเลข (แปลง 'nan' เป็น NaN จริงๆ)
df_std['GPA'] = pd.to_numeric(df_std['GPA'], errors='coerce')

# แทนที่ค่าผิดปกติ (-1.00) ด้วย NaN ก่อนคำนวณค่าเฉลี่ย
df_std['GPA'] = df_std['GPA'].replace(-1.00, np.nan)

# คำนวณค่าเฉลี่ยเฉพาะค่าที่ถูกต้อง (ไม่รวม NaN)
mean_GPA = df_std['GPA'].mean()

# เติมค่าที่เป็น NaN ด้วยค่าเฉลี่ย
df_std['GPA'].fillna(mean_GPA, inplace=True)

df_std

,ชื่อ,GPA
0,ธีระ,3.78000
1,ยุพา,3.23000
2,สมชาย,3.21000
3,มานี,3.01000
4,บุญมี,3.23000
5,ภูมิ,3.89000
6,ลัดดา,3.45000
7,สมใจ,3.89000
8,นภา,3.46125


## วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด
### ตัวอย่างข้อมูลนักศึกษาที่มีส่วนสูงที่หายไป

In [16]:
# Recreate the original df_std DataFrame
df_std = pd.DataFrame(student_data)

# Add the 'วันที่ลงทะเบียน' column back
base_date = datetime(2024, 1, 15)
df_std['วันที่ลงทะเบียน'] = [base_date + timedelta(days=np.random.randint(-30, 30))
                                for _ in range(20)]

l_data = {
    'ชื่อ' : ['ธีระ', 'ยุพา', 'สมชาย' , 'มานี' , 'บุญมี', 'สมศักดิ์' , 'ภูมิ', 'ลัดดา' , 'สมใจ' , 'นภา'],
    'ส่วนสูง_cm' : ['NaN', '161.0', '170.0', '166.0', '169.0', '168.0' , '173.0' , '168.0' , '160.0' , '171.0' ]
}
df_std = pd.DataFrame(l_data)

# แปลงค่าในคอลัมน์ ส่วนสูง ให้เป็นตัวเลข (แปลง 'nan' เป็น NaN จริงๆ)
df_std['ส่วนสูง_cm'] = pd.to_numeric(df_std['ส่วนสูง_cm'], errors='coerce')

# คำนวณค่าเฉลี่ยเฉพาะค่าที่ถูกต้อง (ไม่รวม NaN)
mean_H = df_std['ส่วนสูง_cm'].mean()

# เติมค่าที่เป็น NaN ด้วยค่าเฉลี่ย
df_std['ส่วนสูง_cm'].fillna(mean_H, inplace=True)

df_std

,ชื่อ,ส่วนสูง_cm
0,ธีระ,167.333333
1,ยุพา,161.000000
2,สมชาย,170.000000
3,มานี,166.000000
4,บุญมี,169.000000
5,สมศักดิ์,168.000000
6,ภูมิ,173.000000
7,ลัดดา,168.000000
8,สมใจ,160.000000
9,นภา,171.000000


## วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด
### ตัวอย่างข้อมูลนักศึกษาที่มีน้ำหนักที่หายไป

In [17]:

# Recreate the original df_std DataFrame
df_std = pd.DataFrame(student_data)

# Add the 'วันที่ลงทะเบียน' column back
base_date = datetime(2024, 1, 15)
df_std['วันที่ลงทะเบียน'] = [base_date + timedelta(days=np.random.randint(-30, 30))
                                for _ in range(20)]

l_data = {
    'ชื่อ' : ['ธีระ', 'ยุพา', 'สมชาย' , 'มานี' , 'บุญมี', 'สมศักดิ์' , 'ภูมิ', 'ลัดดา' , 'สมใจ' , 'นภา'],
    'น้ำหนัก_kg' : ['68.0', '50.0', '65.0', '62.0', '72.0', '70.0' , '78.0' , '64.0' , '52.0' , 'NaN']
}
df_std = pd.DataFrame(l_data)

# แปลงค่าในคอลัมน์ น้ำหนัก ให้เป็นตัวเลข (แปลง 'nan' เป็น NaN จริงๆ)
df_std['น้ำหนัก_kg'] = pd.to_numeric(df_std['น้ำหนัก_kg'], errors='coerce')

# คำนวณค่าเฉลี่ยเฉพาะค่าที่ถูกต้อง (ไม่รวม NaN)
mean_W = df_std['น้ำหนัก_kg'].mean()

# เติมค่าที่เป็น NaN ด้วยค่าเฉลี่ย
df_std['น้ำหนัก_kg'].fillna(mean_W, inplace=True)

df_std

,ชื่อ,น้ำหนัก_kg
0,ธีระ,68.000000
1,ยุพา,50.000000
2,สมชาย,65.000000
3,มานี,62.000000
4,บุญมี,72.000000
5,สมศักดิ์,70.000000
6,ภูมิ,78.000000
7,ลัดดา,64.000000
8,สมใจ,52.000000
9,นภา,64.555556


## วิธีที่ 3 เติมค่าด้วยค่าที่พบบ่อยที่สุด
### ตัวอย่างข้อมูลนักศึกษาที่มีชั่วโมงเรียนต่อสัปดาห์ที่หายไป


In [18]:
# Recreate the original df_std DataFrame
df_std = pd.DataFrame(student_data)

# Add the 'วันที่ลงทะเบียน' column back
base_date = datetime(2024, 1, 15)
df_std['วันที่ลงทะเบียน'] = [base_date + timedelta(days=np.random.randint(-30, 30))
                                for _ in range(20)]

l_data = {
    'ชื่อ' : ['ธีระ', 'ยุพา', 'สมชาย' , 'มานี' , 'บุญมี', 'สมศักดิ์' , 'ภูมิ', 'ลัดดา' , 'สมใจ' , 'นภา'],
    'ชั่วโมงเรียน_ต่อสัปดาห์' : ['NaN', '27.0', '15.0', '21.0', '24.0', '10.0' , '14.0' , '23.0' , '30.0' , '16.0']
}
df_std = pd.DataFrame(l_data)

# แปลงค่าในคอลัมน์ น้ำหนัก ให้เป็นตัวเลข (แปลง 'nan' เป็น NaN จริงๆ)
df_std['ชั่วโมงเรียน_ต่อสัปดาห์'] = pd.to_numeric(df_std['ชั่วโมงเรียน_ต่อสัปดาห์'], errors='coerce')

# คำนวณค่าเฉลี่ยเฉพาะค่าที่ถูกต้อง (ไม่รวม NaN)
mean_Ho = df_std['ชั่วโมงเรียน_ต่อสัปดาห์'].mean()

# เติมค่าที่เป็น NaN ด้วยค่าเฉลี่ย
df_std['ชั่วโมงเรียน_ต่อสัปดาห์'].fillna(mean_Ho, inplace=True)

df_std

,ชื่อ,ชั่วโมงเรียน_ต่อสัปดาห์
0,ธีระ,20.0
1,ยุพา,27.0
2,สมชาย,15.0
3,มานี,21.0
4,บุญมี,24.0
5,สมศักดิ์,10.0
6,ภูมิ,14.0
7,ลัดดา,23.0
8,สมใจ,30.0
9,นภา,16.0


#3.2 การทำให้ข้อมูลเป็นมาตรฐาน
### ทำให้ข้อมูลเป็นมาตราฐานเดียวกัน (เพศ)


In [19]:
# Recreate the original df_std DataFrame
df_std = pd.DataFrame(student_data)

# Add the 'วันที่ลงทะเบียน' column back
base_date = datetime(2024, 1, 15)
df_std['วันที่ลงทะเบียน'] = [base_date + timedelta(days=np.random.randint(-30, 30))
                                for _ in range(20)]


def standardize_sex(name):
  if name in ['หญิง', 'ญ' , 'F' , 'Female']:
    return 'หญิง'
  elif name in['ชาย', 'M', 'ช', 'Male' ]:
    return 'ชาย'
  # Keep NaN values as they are
  elif pd.isna(name):
    return np.nan
  else:
    return name

df_std['เพศ_มาตรฐาน'] = df_std['เพศ'].apply(standardize_sex)
df_std

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน,เพศ_มาตรฐาน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-01-24,ชาย
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.45,165.0,58.0,20.0,2023-12-19,หญิง
2,64003,None,NaN,Male,Science,3.67,175.0,NaN,25.0,2023-12-17,ชาย
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2023-12-21,ชาย
4,64005,สมใจ,19.0,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-02-07,หญิง
5,64006,กานต์,150.0,ชาย,Engineering,3.12,172.0,75.0,12.0,2024-01-26,ชาย
6,64007,กิตติ,21.0,หญิง,วิศวกรรม,4.50,158.0,48.0,35.0,2023-12-19,หญิง
7,64008,จิรา,20.0,F,ศิลปศาสตร์,2.98,180.0,82.0,18.0,2024-02-07,หญิง
8,64009,ชนาธิป,23.0,M,Arts,3.34,163.0,55.0,22.0,2024-01-13,ชาย
9,64010,ดารา,21.0,หญิง,ศิลป์,3.56,167.0,60.0,28.0,2024-01-02,หญิง


###ทำให้ข้อมูลเป็นมาตราฐานเดียวกัน (คณะ)

In [20]:
def standardize_faculty(name):
  if name in ['วิทยาศาสตร์', 'วิทย์', 'Science', 'คณะวิทยาศาสตร์', 'Sci']:
        return 'วิทยาศาสตร์'
  elif name in ['ศิลปศาสตร์', 'ศิลป์', 'Arts', 'Liberal Arts']:
        return 'ศิลปศาสตร์'
  elif name in ['Engineering', 'วิศวกรรม', 'วิศวฯ', 'วิศวะ']:
        return 'วิศวกรรมศาสตร์'
  else:
        return name

df_std['คณะ_มาตรฐาน'] = df_std['คณะ'].apply(standardize_faculty)
df_std

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน,เพศ_มาตรฐาน,คณะ_มาตรฐาน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-01-24,ชาย,วิทยาศาสตร์
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.45,165.0,58.0,20.0,2023-12-19,หญิง,วิทยาศาสตร์
2,64003,None,NaN,Male,Science,3.67,175.0,NaN,25.0,2023-12-17,ชาย,วิทยาศาสตร์
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2023-12-21,ชาย,วิทยาศาสตร์
4,64005,สมใจ,19.0,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-02-07,หญิง,วิศวกรรมศาสตร์
5,64006,กานต์,150.0,ชาย,Engineering,3.12,172.0,75.0,12.0,2024-01-26,ชาย,วิศวกรรมศาสตร์
6,64007,กิตติ,21.0,หญิง,วิศวกรรม,4.50,158.0,48.0,35.0,2023-12-19,หญิง,วิศวกรรมศาสตร์
7,64008,จิรา,20.0,F,ศิลปศาสตร์,2.98,180.0,82.0,18.0,2024-02-07,หญิง,ศิลปศาสตร์
8,64009,ชนาธิป,23.0,M,Arts,3.34,163.0,55.0,22.0,2024-01-13,ชาย,ศิลปศาสตร์
9,64010,ดารา,21.0,หญิง,ศิลป์,3.56,167.0,60.0,28.0,2024-01-02,หญิง,ศิลปศาสตร์


In [21]:
# สร้างสำเนาของ DataFrame df_original
df_cleaned = df_original.copy()

# เติมค่าที่หายไปในคอลัมน์ตัวเลขด้วยค่าเฉลี่ย
numeric_cols = ['อายุ', 'ส่วนสูง_cm', 'น้ำหนัก_kg', 'ชั่วโมงเรียน_ต่อสัปดาห์']
for col in numeric_cols:
    mean_value = df_cleaned[col].mean()
    df_cleaned[col].fillna(mean_value, inplace=True)

# เติมค่าที่หายไปในคอลัมน์ข้อมูลประเภทด้วยค่าฐานนิยม
categorical_cols = ['ชื่อ', 'เพศ', 'คณะ']
for col in categorical_cols:
    # Handle cases where mode might return an empty Series (e.g., all NaNs)
    if not df_cleaned[col].mode().empty:
        most_common_value = df_cleaned[col].mode()[0]
        df_cleaned[col].fillna(most_common_value, inplace=True)
    else:
        # If all values are NaN, fill with a placeholder or leave as NaN
        # For this task, we'll leave them as NaN if mode is empty
        pass


# กำหนดฟังก์ชัน standardized_sex
def standardize_sex(name):
  if pd.isna(name):
    return np.nan # Keep NaN values as they are
  elif name in ['หญิง', 'ญ' , 'F' , 'Female']:
    return 'หญิง'
  elif name in['ชาย', 'M', 'ช', 'Male' ]:
    return 'ชาย'
  else:
    return name # Keep other values as they are


# กำหนดฟังก์ชัน standardize_faculty
def standardize_faculty(name):
  if pd.isna(name):
    return np.nan # Keep NaN values as they are
  elif name in ['วิทยาศาสตร์', 'วิทย์', 'Science', 'คณะวิทยาศาสตร์', 'Sci']:
        return 'วิทยาศาสตร์'
  elif name in ['ศิลปศาสตร์', 'ศิลป์', 'Arts', 'Liberal Arts']:
        return 'ศิลปศาสตร์'
  elif name in ['Engineering', 'วิศวกรรม', 'วิศวฯ', 'วิศวะ', 'วิศวกรรมศาสตร์']:
        return 'วิศวกรรมศาสตร์'
  else:
        return name # Keep other values as they are

# ใช้ฟังก์ชัน standardize_sex เพื่อสร้างคอลัมน์ เพศ_มาตรฐาน
df_cleaned['เพศ_มาตรฐาน'] = df_cleaned['เพศ'].apply(standardize_sex)

# ใช้ฟังก์ชัน standardize_faculty เพื่อสร้างคอลัมน์ คณะ_มาตรฐาน
df_cleaned['คณะ_มาตรฐาน'] = df_cleaned['คณะ'].apply(standardize_faculty)

# แสดง DataFrame ที่ทำความสะอาดแล้ว
display(df_cleaned)

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน,เพศ_มาตรฐาน,คณะ_มาตรฐาน
0,64001,สมชาย,20.000000,ชาย,วิทยาศาสตร์,3.21,170.000000,65.000000,15.000000,2024-01-23,ชาย,วิทยาศาสตร์
1,64002,สมหญิง,21.000000,หญิง,วิทย์,3.45,165.000000,58.000000,20.000000,2024-02-05,หญิง,วิทยาศาสตร์
2,64003,กานต์,28.470588,Male,Science,3.67,175.000000,63.611111,25.000000,2024-01-13,ชาย,วิทยาศาสตร์
3,64004,สมศักดิ์,22.000000,ช,คณะวิทยาศาสตร์,-1.00,168.000000,70.000000,10.000000,2023-12-30,ชาย,วิทยาศาสตร์
4,64005,สมใจ,19.000000,ญ,วิศวะ,3.89,160.000000,52.000000,30.000000,2024-01-27,หญิง,วิศวกรรมศาสตร์
5,64006,กานต์,150.000000,ชาย,Engineering,3.12,172.000000,75.000000,12.000000,2023-12-23,ชาย,วิศวกรรมศาสตร์
6,64007,กิตติ,21.000000,หญิง,วิศวกรรม,4.50,158.000000,48.000000,35.000000,2024-01-05,หญิง,วิศวกรรมศาสตร์
7,64008,จิรา,20.000000,F,ศิลปศาสตร์,2.98,180.000000,82.000000,18.000000,2024-01-23,หญิง,ศิลปศาสตร์
8,64009,ชนาธิป,23.000000,M,Arts,3.34,163.000000,55.000000,22.000000,2024-02-11,ชาย,ศิลปศาสตร์
9,64010,ดารา,21.000000,หญิง,ศิลป์,3.56,167.000000,60.000000,28.000000,2024-01-03,หญิง,ศิลปศาสตร์


#3.3 การจัดการค่าผิดปกติ
##จัดการค่าผิดปกติของอายุ และ GPA

In [22]:
import pandas as pd
import numpy as np

# สร้างข้อมูลตัวอย่าง
data = {
    'อายุ': [20, 21, 22, 19, 20, 150, 21, 22, 23, 18],
    'GPA': [3.78, 3.23, 3.21, -1.0, 3.23, 4.5, 3.89, 3.45, 3.89, 5.0]
}
df = pd.DataFrame(data)

def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    #กำหนดขอบเขต
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    return df[(df[column] < lower) | (df[column] > upper)]

# ตรวจจับค่าผิดปกติ
print("ค่าผิดปกติในคอลัมน์ 'อายุ':")
print(detect_outliers_iqr(df, 'อายุ'))

print("\nค่าผิดปกติในคอลัมน์ 'GPA':")
print(detect_outliers_iqr(df, 'GPA'))

def handle_outliers(df, column, method='mean'):
    # คำนวณขอบเขต IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR

    # คัดลอก DataFrame เพื่อป้องกันการแก้ไขข้อมูลต้นฉบับ
    df_clean = df.copy()

    # วิธีการจัดการ (เลือกได้ 3 วิธี)
    if method == 'mean':
        # แทนที่ด้วยค่าเฉลี่ยของข้อมูลปกติ
        mean_val = df[(df[column] >= lower) & (df[column] <= upper)][column].mean()
        df_clean.loc[(df[column] < lower) | (df[column] > upper), column] = mean_val
    elif method == 'median':
        # แทนที่ด้วยค่ามัธยฐานของข้อมูลปกติ
        median_val = df[(df[column] >= lower) & (df[column] <= upper)][column].median()
        df_clean.loc[(df[column] < lower) | (df[column] > upper), column] = median_val
    elif method == 'remove':
        # ลบแถวที่มีค่าผิดปกติ
        df_clean = df[(df[column] >= lower) & (df[column] <= upper)].copy()

    return df_clean

# ตัวอย่างการใช้งาน
df_clean_mean = handle_outliers(df, 'อายุ', method='mean')
df_clean_median = handle_outliers(df, 'GPA', method='median')
df_clean_remove = handle_outliers(df, 'อายุ', method='remove')

print("\nผลลัพธ์หลังจัดการด้วยค่าเฉลี่ย:")
print(df_clean_mean)

print("\nผลลัพธ์หลังจัดการด้วยค่ามัธยฐาน:")
print(df_clean_median)

print("\nผลลัพธ์หลังลบค่าผิดปกติ:")
print(df_clean_remove)

ค่าผิดปกติในคอลัมน์ 'อายุ':
   อายุ  GPA
5   150  4.5

ค่าผิดปกติในคอลัมน์ 'GPA':
   อายุ  GPA
3    19 -1.0
9    18  5.0

ผลลัพธ์หลังจัดการด้วยค่าเฉลี่ย:
        อายุ   GPA
0  20.000000  3.78
1  21.000000  3.23
2  22.000000  3.21
3  19.000000 -1.00
4  20.000000  3.23
5  20.666667  4.50
6  21.000000  3.89
7  22.000000  3.45
8  23.000000  3.89
9  18.000000  5.00

ผลลัพธ์หลังจัดการด้วยค่ามัธยฐาน:
   อายุ    GPA
0    20  3.780
1    21  3.230
2    22  3.210
3    19  3.615
4    20  3.230
5   150  4.500
6    21  3.890
7    22  3.450
8    23  3.890
9    18  3.615

ผลลัพธ์หลังลบค่าผิดปกติ:
   อายุ   GPA
0    20  3.78
1    21  3.23
2    22  3.21
3    19 -1.00
4    20  3.23
6    21  3.89
7    22  3.45
8    23  3.89
9    18  5.00


#4.1 One-Hot Encoding แปลงข้อมูลประเภทเป็นตัวเลข


In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display  # เพิ่มการ import นี้ถ้าใช้ใน Jupyter Notebook

# สร้าง DataFrame จากข้อมูลของคุณ
data = {
    'รหัสนักศึกษา': ['64001', '64002', '64003', '64004'],
    'คณะ': ['วิทยาศาสตร์', 'วิศวกรรมศาสตร์', 'ศิลปศาสตร์', 'วิทยาศาสตร์'],
    'เพศ': ['ชาย', 'หญิง', 'ชาย', 'หญิง']
}
df = pd.DataFrame(data)

# สร้าง encoder และแปลงข้อมูล
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = encoder.fit_transform(df[['คณะ', 'เพศ']])

# ได้ชื่อคอลัมน์แบบเดียวกับใน Colab
new_columns = []
for i, col in enumerate(['คณะ', 'เพศ']):
    categories = encoder.categories_[i][1:]  # ลบ category แรกออก (drop_first)
    new_columns.extend([f"{col}_{cat}" for cat in categories])

# สร้าง DataFrame ใหม่
df_encoded = pd.DataFrame(encoded_data, columns=new_columns)
df_final = pd.concat([df[['รหัสนักศึกษา']], df_encoded], axis=1)

# แสดงผลลัพธ์
display(df_final)  # แก้ไขจาก display(df_final) ดูโค๊ดที่เกิดการ Error ให้หน่อย แล้วควรแก้ไขอย่างไร

,รหัสนักศึกษา,คณะ_วิศวกรรมศาสตร์,คณะ_ศิลปศาสตร์,เพศ_หญิง
0,64001,0.0,0.0,0.0
1,64002,1.0,0.0,1.0
2,64003,0.0,1.0,0.0
3,64004,0.0,0.0,1.0


#4.2 การสร้าง Feature จากวันเวลา

In [24]:
import pandas as pd
import numpy as np
from datetime import datetime

# สร้างข้อมูลนักศึกษาแบบเดียวกับโครงสร้างใน Colab
data = {
    'student_id': [f'ST{str(i).zfill(3)}' for i in range(1, 11)],
    'enroll_date': pd.to_datetime([
        '2025-08-15 09:30', '2025-08-16 13:45', '2025-08-17 15:20',
        '2025-08-18 11:10', '2025-08-19 14:30', '2025-08-20 10:15',
        '2025-08-21 08:45', '2025-08-22 16:20', '2025-08-23 12:00',
        '2025-08-24 09:30'
    ]),
    'activity_score': np.random.randint(70, 100, size=10)
}
df = pd.DataFrame(data)
display(df)

,student_id,enroll_date,activity_score
0,ST001,2025-08-15 09:30:00,83
1,ST002,2025-08-16 13:45:00,92
2,ST003,2025-08-17 15:20:00,97
3,ST004,2025-08-18 11:10:00,94
4,ST005,2025-08-19 14:30:00,99
5,ST006,2025-08-20 10:15:00,77
6,ST007,2025-08-21 08:45:00,90
7,ST008,2025-08-22 16:20:00,85
8,ST009,2025-08-23 12:00:00,82
9,ST010,2025-08-24 09:30:00,87


# 4.3 การรวม Features

In [25]:
import pandas as pd
import numpy as np

# สร้างข้อมูลตัวอย่างจาก student_data ของคุณ
data = {
    'student_id': ['ST001', 'ST002', 'ST003', 'ST004', 'ST005',
                  'ST006', 'ST007', 'ST008', 'ST009', 'ST010'],
    'faculty': ['Science', 'Engineering', 'Science', 'Arts', 'Engineering',
               'Science', 'Arts', 'Engineering', 'Science', 'Arts'],
    'gpa': [3.2, 3.5, 3.8, 3.1, 3.4, 3.9, 3.0, 3.6, 3.7, 3.3],
    'activity_score': [85, 92, 78, 88, 90, 82, 95, 87, 84, 91],
    'enrollment_week': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]
}
df = pd.DataFrame(data)

# 1.1 การรวมด้วย GroupBy (เหมือนใน Colab)
faculty_stats = df.groupby('faculty').agg({
    'gpa': ['mean', 'median', 'count'],
    'activity_score': ['mean', 'max', 'min'],
    'enrollment_week': 'std'  # ความแปรปรวนของสัปดาห์ลงทะเบียน
}).round(2)

# ตั้งชื่อคอลัมน์ใหม่แบบใน Colab
faculty_stats.columns = [
    'gpa_mean', 'gpa_median', 'student_count',
    'activity_mean', 'activity_max', 'activity_min',
    'enrollment_week_std'
]

print("=== สถิติรวมตามคณะ ===")
print(faculty_stats)

# 2.1 สร้าง Interaction ระหว่าง GPA และ Activity Score (เหมือนใน Colab ที่รวม Horsepower และ Weight)
df['gpa_activity_interaction'] = df['gpa'] * df['activity_score']

# 2.2 สร้าง Ratio Feature
df['activity_per_week'] = df['activity_score'] / df['enrollment_week']

print("\n=== ข้อมูลหลังสร้าง Interaction Features ===")
print(df[['student_id', 'gpa', 'activity_score',
         'gpa_activity_interaction', 'activity_per_week']].head())

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# เลือก Features ตัวเลข
numeric_features = df[['gpa', 'activity_score', 'enrollment_week']]

# มาตรฐาน化ข้อมูล (เหมือนใน Colab)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_features)

# ใช้ PCA ลดมิติ (เหมือนใน Colab)
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_data)

# เพิ่มผลลัพธ์ลงใน DataFrame
df['pca1'] = principal_components[:, 0]
df['pca2'] = principal_components[:, 1]

print("\n=== ผลลัพธ์ PCA ===")
print("อธิบายความแปรปรวน:", pca.explained_variance_ratio_)
print(df[['student_id', 'pca1', 'pca2']].head())

# สร้าง Feature Cross ระหว่างคณะและสัปดาห์ลงทะเบียน
df['faculty_week_cross'] = df['faculty'] + "_week" + df['enrollment_week'].astype(str)

# One-Hot Encoding สำหรับ Feature Cross (เหมือนใน Colab)
cross_encoded = pd.get_dummies(df['faculty_week_cross'], prefix='cross')

# รวมกับ DataFrame หลัก
df = pd.concat([df, cross_encoded], axis=1)

print("\n=== ข้อมูลหลังสร้าง Feature Cross ===")
print(df[['student_id', 'faculty', 'enrollment_week', 'faculty_week_cross']].head())

# คำนวณค่าเฉลี่ย GPA ของแต่ละคณะ
faculty_gpa_mean = df.groupby('faculty')['gpa'].transform('mean')

# สร้าง Feature ใหม่: ส่วนต่างจากค่าเฉลี่ยคณะ
df['gpa_deviation'] = df['gpa'] - faculty_gpa_mean

print("\n=== ข้อมูลส่วนต่าง GPA จากค่าเฉลี่ยคณะ ===")
print(df[['student_id', 'faculty', 'gpa', 'gpa_deviation']].sort_values('faculty'))

=== สถิติรวมตามคณะ ===
             gpa_mean  gpa_median  student_count  activity_mean  activity_max  \
faculty                                                                         
Arts             3.13        3.10              3          91.33            95   
Engineering      3.50        3.50              3          89.67            92   
Science          3.65        3.75              4          82.25            85   

             activity_min  enrollment_week_std  
faculty                                         
Arts                   88                 1.53  
Engineering            87                 1.53  
Science                78                 1.71  

=== ข้อมูลหลังสร้าง Interaction Features ===
  student_id  gpa  activity_score  gpa_activity_interaction  activity_per_week
0      ST001  3.2              85                     272.0               85.0
1      ST002  3.5              92                     322.0               92.0
2      ST003  3.8              78          

#รวมผลลัพธ์ทั้งหมด

In [26]:
# 1: รวมขั้นตอนการจัดการข้อมูลหายและทำให้เป็นมาตรฐาน

# ทำการสร้างสำเนาของ DataFrame ต้นฉบับ
df_combined_cleaned = df_original.copy()

# เติมค่าที่หายไปในคอลัมน์ตัวเลขด้วยค่าเฉลี่ย
numeric_cols = ['อายุ', 'ส่วนสูง_cm', 'น้ำหนัก_kg', 'ชั่วโมงเรียน_ต่อสัปดาห์']
for col in numeric_cols:
    mean_value = df_combined_cleaned[col].mean()
    df_combined_cleaned[col].fillna(mean_value, inplace=True)

# เติมค่าที่หายไปในคอลัมน์ข้อมูลด้วยค่าฐานนิยม
categorical_cols = ['ชื่อ', 'เพศ', 'คณะ']
for col in categorical_cols:

    # จัดการกรณีที่โหมดอาจส่งคืน Series ที่ว่างเปล่า (เช่น NaN ทั้งหมด)
    if not df_combined_cleaned[col].mode().empty:
        most_common_value = df_combined_cleaned[col].mode()[0]
        df_combined_cleaned[col].fillna(most_common_value, inplace=True)
    else:
        # ถ้าค่าทั้งหมดเป็น NaN ให้เติมด้วยตัวแทน หรือปล่อยไว้เป็น NaN
        # สำหรับงานนี้ เราจะเติมด้วยตัวแทน 'ไม่ทราบ'
        df_combined_cleaned[col].fillna('Unknown', inplace=True)


# กำหนดฟังก์ชัน standardize_sex
def standardize_sex(name):
  if pd.isna(name):
    return np.nan # Keep NaN values as they are
  elif name in ['หญิง', 'ญ' , 'F' , 'Female']:
    return 'หญิง'
  elif name in['ชาย', 'M', 'ช', 'Male' ]:
    return 'ชาย'
  else:
    return name # Keep other values as they are


# กำหนดฟังก์ชัน standardize_faculty
def standardize_faculty(name):
  if pd.isna(name):
    return np.nan # Keep NaN values as they are
  elif name in ['วิทยาศาสตร์', 'วิทย์', 'Science', 'คณะวิทยาศาสตร์', 'Sci']:
        return 'วิทยาศาสตร์'
  elif name in ['ศิลปศาสตร์', 'ศิลป์', 'Arts', 'Liberal Arts']:
        return 'ศิลปศาสตร์'
  elif name in ['Engineering', 'วิศวกรรม', 'วิศวฯ', 'วิศวะ', 'วิศวกรรมศาสตร์']:
        return 'วิศวกรรมศาสตร์'
  else:
        return name


# ใช้ฟังก์ชัน standardize_sex เพื่อสร้างคอลัมน์ เพศ_มาตรฐาน
df_combined_cleaned['เพศ_มาตรฐาน'] = df_combined_cleaned['เพศ'].apply(standardize_sex)

# ใช้ฟังก์ชัน standardize_faculty เพื่อสร้างคอลัมน์ คณะ_มาตรฐาน
df_combined_cleaned['คณะ_มาตรฐาน'] = df_combined_cleaned['คณะ'].apply(standardize_faculty)

# 2: จัดการค่าผิดปกติ (ใช้ IQR และแทนที่ด้วยค่ามัธยฐานสำหรับตัวเลข)

def handle_outliers_median(df, column):
    # คำนวณขอบเขต IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR

    # คัดลอก DataFrame เพื่อป้องกันการแก้ไขข้อมูลต้นฉบับ
    df_clean = df.copy()

    # แทนที่ด้วยค่ามัธยฐานของข้อมูลปกติ
    median_val = df[(df[column] >= lower) & (df[column] <= upper)][column].median()
    df_clean.loc[(df[column] < lower) | (df[column] > upper), column] = median_val

    return df_clean

# จัดการค่าผิดปกติในคอลัมน์ 'อายุ'
df_combined_cleaned = handle_outliers_median(df_combined_cleaned, 'อายุ')

# จัดการค่าผิดปกติในคอลัมน์ 'GPA' (GPA ควรมีค่าระหว่าง 0-4, ใช้การกรองตรงๆ แทน IQR ที่อาจไม่เหมาะสมกับ min=-1, max=4.5)
df_combined_cleaned.loc[(df_combined_cleaned['GPA'] < 0) | (df_combined_cleaned['GPA'] > 4), 'GPA'] = np.nan

# หลังจากจัดการค่าผิดปกติใน GPA แล้ว อาจมีค่า NaN เกิดขึ้นอีกครั้ง
# เติมค่า NaN ที่อาจเกิดขึ้นจากการจัดการค่าผิดปกติใน GPA ด้วยค่าเฉลี่ยอีกครั้ง
mean_gpa_after_outlier_handling = df_combined_cleaned['GPA'].mean()
df_combined_cleaned['GPA'].fillna(mean_gpa_after_outlier_handling, inplace=True)


# ขั้นตอนที่ 3: แสดงผลลัพธ์
print("DataFrame หลังจากรวมทุกขั้นตอนการทำความสะอาดข้อมูล:")
display(df_combined_cleaned)

DataFrame หลังจากรวมทุกขั้นตอนการทำความสะอาดข้อมูล:


,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน,เพศ_มาตรฐาน,คณะ_มาตรฐาน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.210000,170.000000,65.000000,15.000000,2024-01-23,ชาย,วิทยาศาสตร์
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.450000,165.000000,58.000000,20.000000,2024-02-05,หญิง,วิทยาศาสตร์
2,64003,กานต์,21.0,Male,Science,3.670000,175.000000,63.611111,25.000000,2024-01-13,ชาย,วิทยาศาสตร์
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,3.433125,168.000000,70.000000,10.000000,2023-12-30,ชาย,วิทยาศาสตร์
4,64005,สมใจ,19.0,ญ,วิศวะ,3.890000,160.000000,52.000000,30.000000,2024-01-27,หญิง,วิศวกรรมศาสตร์
5,64006,กานต์,21.0,ชาย,Engineering,3.120000,172.000000,75.000000,12.000000,2023-12-23,ชาย,วิศวกรรมศาสตร์
6,64007,กิตติ,21.0,หญิง,วิศวกรรม,3.433125,158.000000,48.000000,35.000000,2024-01-05,หญิง,วิศวกรรมศาสตร์
7,64008,จิรา,20.0,F,ศิลปศาสตร์,2.980000,180.000000,82.000000,18.000000,2024-01-23,หญิง,ศิลปศาสตร์
8,64009,ชนาธิป,23.0,M,Arts,3.340000,163.000000,55.000000,22.000000,2024-02-11,ชาย,ศิลปศาสตร์
9,64010,ดารา,21.0,หญิง,ศิลป์,3.560000,167.000000,60.000000,28.000000,2024-01-03,หญิง,ศิลปศาสตร์
